# Implementation of Finite Mixture Model

## Import Needed Filepaths and Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from texas_gerrymandering_hb4.config import FINAL_CSV

## Load Dataset Into Pandas DataFrame

In [ ]:
df = pd.read_csv(FINAL_CSV)

## Define Response Variable `y`

In [ ]:
y = df["dem_share"].values.astype(float)

## Set Up Reproducible Random Number Generator

In [ ]:
np.random.seed(123)

## Setting Up Mixture Model

## Initial Parameters
* These parameters represent starting guesses for the Gibbs sampler.
* `lambda` is the mixture weight. It is set to 0.5 because we are assuming half the data and the other half .

In [ ]:
lambda = 0.5
mu_1 = np.mean(y)
mu_2 = np.mean(y)
sigma_squared_1 = np.var(y)
sigma_squared_2 = np.var(y)

### Setting Priors

In [ ]:
alpha_1 = 2
alpha_2 = 2